## SLOs for Week 02

At the end of this unit, students will be able to...

1.  Perform data aggregation using GROUP BY and HAVING.
2.  Write and execute INNER, LEFT-OUTER, RIGHT, and FULL JOINs.
3.  Construct subqueries for complex queries.
4.  Implement SQL functions and expressions.
5.  Use SQL best practices for readable and efficient queries.
6.  Optimize SQL queries for better performance.

## 0. Creating the Library Survey Tables

In the US, the Institute of Museum and Library Services (IMLS) measures library activity as part of its annual Public Libraries Survey. The survey collects data from more than 9000 library administrative entities, defined by the survey as agencies that provide library services to a particular locality. Data includes the number of branches, staff, books, hours open per year, etc. To teach the concepts below, we will build three tables containing the data from the survey related to the years of 2016, 2017 and 2018. For doing so, we read from the CSV files downloaded from their website. More especifically, some columns will be selected in the process to reduce the amount of non used attributes.

We are running SQL queries in a Jupyter environment. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///dbs/w02/library.db
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

This will open our database `library.db` for us. Don't bother yourself with the config details. That is a trick to run the SQL queries in this environment.

Now we turn to our tables. We have to create 3 tables. Let's start with the table for 2018.

In [2]:
%%sql
-- Creating the 2018 Public Libraries Survey table

-- We drop an old copy of the table, if it exists.
DROP TABLE IF EXISTS libraries_2018;

CREATE TABLE libraries_2018 (
    stabr text NOT NULL,
    fscskey text CONSTRAINT fscskey_2018_pkey PRIMARY KEY,
    libid text NOT NULL,
    libname text NOT NULL,
    address text NOT NULL,
    city text NOT NULL,
    zip text NOT NULL,
    county text NOT NULL,
    phone text NOT NULL,
    c_relatn text NOT NULL,
    c_legbas text NOT NULL,
    c_admin text NOT NULL,
    c_fscs text NOT NULL,
    geocode text NOT NULL,
    lsabound text NOT NULL,
    startdate text NOT NULL,
    enddate text NOT NULL,
    popu_lsa integer NOT NULL,
    popu_und integer NOT NULL,
    centlib integer NOT NULL,
    branlib integer NOT NULL,
    bkmob integer NOT NULL,
    totstaff numeric(8,2) NOT NULL,
    bkvol integer NOT NULL,
    ebook integer NOT NULL,
    audio_ph integer NOT NULL,
    audio_dl integer NOT NULL,
    video_ph integer NOT NULL,
    video_dl integer NOT NULL,
    ec_lo_ot integer NOT NULL,
    subscrip integer NOT NULL,
    hrs_open integer NOT NULL,
    visits integer NOT NULL,
    reference integer NOT NULL,
    regbor integer NOT NULL,
    totcir integer NOT NULL,
    kidcircl integer NOT NULL,
    totpro integer NOT NULL,
    gpterms integer NOT NULL,
    pitusr integer NOT NULL,
    wifisess integer NOT NULL,
    obereg text NOT NULL,
    statstru text NOT NULL,
    statname text NOT NULL,
    stataddr text NOT NULL,
    longitude numeric(10,7) NOT NULL,
    latitude numeric(10,7) NOT NULL
);

 * sqlite:///dbs/w02/library.db
Done.
Done.


[]

**This is** an empty table. To fill the table, we need to convert our CSV entries to entitites in our database.

Here I came up with another trick. I created a subprocess that opens SQLite3 and calls the command to convert a CSV file into a table inside the database. The code is as follows:

In [3]:
import subprocess

db = "dbs/w02/library.db"
csv = "dbs/w02/pls_fy2018_libraries.csv"
table = "libraries_2018"

sqlite_command = f"""
.mode csv
.import '{csv}' {table}
"""

subprocess.run(["sqlite3", db], input=sqlite_command, text=True)

CompletedProcess(args=['sqlite3', 'dbs/w02/library.db'], returncode=0)

**Let's see** if the data was loaded? Let's peak on the 10 first rows.

In [4]:
%%sql
SELECT * FROM libraries_2018
LIMIT 10;

 * sqlite:///dbs/w02/library.db
Done.


stabr,fscskey,libid,libname,address,city,zip,county,phone,c_relatn,c_legbas,c_admin,c_fscs,geocode,lsabound,startdate,enddate,popu_lsa,popu_und,centlib,branlib,bkmob,totstaff,bkvol,ebook,audio_ph,audio_dl,video_ph,video_dl,ec_lo_ot,subscrip,hrs_open,visits,reference,regbor,totcir,kidcircl,totpro,gpterms,pitusr,wifisess,obereg,statstru,statname,stataddr,longitude,latitude
STABR,FSCSKEY,LIBID,LIBNAME,ADDRESS,CITY,ZIP,CNTY,PHONE,C_RELATN,C_LEGBAS,C_ADMIN,C_FSCS,GEOCODE,LSABOUND,STARTDAT,ENDDATE,POPU_LSA,POPU_UND,CENTLIB,BRANLIB,BKMOB,TOTSTAFF,BKVOL,EBOOK,AUDIO_PH,AUDIO_DL,VIDEO_PH,VIDEO_DL,EC_LO_OT,SUBSCRIP,HRS_OPEN,VISITS,REFERENC,REGBOR,TOTCIR,KIDCIRCL,TOTPRO,GPTERMS,PITUSR,WIFISESS,OBEREG,STATSTRU,STATNAME,STATADDR,LONGITUD,LATITUDE
AK,AK0001,AK0001-002,ANCHOR POINT PUBLIC LIBRARY,34020 NORTH FORK ROAD,ANCHOR POINT,99556,KENAI PENINSULA,9072355692,NO,NP,SO,Y,CI1,N,7/1/17,6/30/18,2057,2040,1,0,0,0.67,18201,0,179,0,3320,0,0,9,1245,6032,4,1726,11316,2294,60,8,1495,1182,08,00,00,00,-151.825,59.77965
AK,AK0002,AK0002-011,ANCHORAGE PUBLIC LIBRARY,3600 DENALI STREET,ANCHORAGE,99503,ANCHORAGE,9073432892,NO,CO,MO,Y,MA1,N,1/1/17,12/31/17,295365,292940,1,4,0,76.65,370812,21587,30963,12992,76404,0,14,7311,11400,723272,54306,135828,1574942,553651,1924,165,126846,90135,08,00,00,00,-149.876,61.18744
AK,AK0003,AK0003-002,ANDERSON COMMUNITY LIBRARY,101 FIRST STREET,ANDERSON,99744,DENALI,9075822628,NO,CI,SO,Y,CI1,N,7/1/17,6/30/18,269,267,1,0,0,0.75,15314,0,201,0,1403,0,0,0,480,592,300,113,1137,264,4,5,225,0,08,00,00,00,-149.187,64.34363
AK,AK0006,AK0006-002,KUSKOKWIM CONSORTIUM LIBRARY,420 CHIEF EDDIE HOFFMAN HIGHWAY,BETHEL,99559,BETHEL,9075434516,NO,MJ,SO,Y,CI1,N,7/1/17,6/30/18,6135,6085,1,0,0,3,34860,21587,425,12992,3782,0,0,5,2040,51000,1530,1912,14067,3746,406,8,6600,5716,08,00,00,00,-161.771,60.79114
AK,AK0007,AK0007-002,BIG LAKE PUBLIC LIBRARY,3140 SOUTH BIG LAKE ROAD,BIG LAKE,99652,MATANUSKA-SUSITNA,9078617635,NO,CO,SO,Y,CO1,N,7/1/17,6/30/18,12847,12742,1,0,0,3,28698,21587,2140,12992,3805,0,0,47,2551,62249,887,2890,36670,15691,363,18,15022,8125,08,00,00,00,-149.819,61.5475
AK,AK0008,AK0008-002,CANTWELL COMMUNITY LIBRARY,1 SCHOOL ROAD,CANTWELL,99729,DENALI,9077682372,NO,NP,SO,Y,CI1,N,7/1/17,6/30/18,202,200,1,0,0,0.8,10376,21587,275,12992,562,0,0,15,720,2500,390,183,2828,1198,45,1,50,95,08,00,00,00,-148.9,63.3912
AK,AK0011,AK0011-002,CHINIAK PUBLIC LIBRARY,43318 SPRUCE WAY,CHINIAK,99615,KODIAK ISLAND,9075120880,NO,NP,SO,N,CI1,N,7/1/17,6/30/18,44,44,1,0,0,0,2300,21587,0,12992,257,0,0,18,132,302,0,39,465,301,27,2,75,220,08,00,00,00,-152.231,57.61245
AK,AK0014,AK0014-002,COLD BAY PUBLIC LIBRARY,10 BARANOV ROAD,COLD BAY,99571,ALEUTIANS EAST,9075322878,NO,NP,SO,Y,CI1,N,7/1/17,6/30/18,63,62,1,0,0,0.25,3226,0,445,0,2673,0,0,0,400,1080,31,36,121,11,5,4,235,893,08,00,00,00,-160.693,55.69687
AK,AK0015,AK0015-002,COOPER LANDING COMMUNITY LIBRARY,18511 BEAN CREEK ROAD,COOPER LANDING,99572,KENAI PENINSULA,9075951241,NO,NP,SO,N,CI1,N,7/1/17,6/30/18,265,263,1,0,0,0,4163,0,192,0,926,0,0,1,1092,2452,70,834,2232,772,11,4,395,357,08,00,00,00,-149.823,60.49404


**Now, lets** count how many rows are in this table.

In [5]:
%%sql
SELECT count(*) FROM libraries_2018;

 * sqlite:///dbs/w02/library.db
Done.


count(*)
9262


**The same** should be done for the years 2016 and 2017. The code below is long, but identical to what we've done above.

In [7]:
%%sql
-- Creating the 2016 Public Libraries Survey table

-- We drop an old copy of the table, if it exists.
DROP TABLE IF EXISTS libraries_2016;
DROP TABLE IF EXISTS libraries_2017;

CREATE TABLE libraries_2016 (
    stabr text NOT NULL,
    fscskey text CONSTRAINT fscskey_2018_pkey PRIMARY KEY,
    libid text NOT NULL,
    libname text NOT NULL,
    address text NOT NULL,
    city text NOT NULL,
    zip text NOT NULL,
    county text NOT NULL,
    phone text NOT NULL,
    c_relatn text NOT NULL,
    c_legbas text NOT NULL,
    c_admin text NOT NULL,
    c_fscs text NOT NULL,
    geocode text NOT NULL,
    lsabound text NOT NULL,
    startdate text NOT NULL,
    enddate text NOT NULL,
    popu_lsa integer NOT NULL,
    popu_und integer NOT NULL,
    centlib integer NOT NULL,
    branlib integer NOT NULL,
    bkmob integer NOT NULL,
    totstaff numeric(8,2) NOT NULL,
    bkvol integer NOT NULL,
    ebook integer NOT NULL,
    audio_ph integer NOT NULL,
    audio_dl integer NOT NULL,
    video_ph integer NOT NULL,
    video_dl integer NOT NULL,
    ec_lo_ot integer NOT NULL,
    subscrip integer NOT NULL,
    hrs_open integer NOT NULL,
    visits integer NOT NULL,
    reference integer NOT NULL,
    regbor integer NOT NULL,
    totcir integer NOT NULL,
    kidcircl integer NOT NULL,
    totpro integer NOT NULL,
    gpterms integer NOT NULL,
    pitusr integer NOT NULL,
    wifisess integer NOT NULL,
    obereg text NOT NULL,
    statstru text NOT NULL,
    statname text NOT NULL,
    stataddr text NOT NULL,
    longitude numeric(10,7) NOT NULL,
    latitude numeric(10,7) NOT NULL
);

CREATE TABLE libraries_2017 (
    stabr text NOT NULL,
    fscskey text CONSTRAINT fscskey_2018_pkey PRIMARY KEY,
    libid text NOT NULL,
    libname text NOT NULL,
    address text NOT NULL,
    city text NOT NULL,
    zip text NOT NULL,
    county text NOT NULL,
    phone text NOT NULL,
    c_relatn text NOT NULL,
    c_legbas text NOT NULL,
    c_admin text NOT NULL,
    c_fscs text NOT NULL,
    geocode text NOT NULL,
    lsabound text NOT NULL,
    startdate text NOT NULL,
    enddate text NOT NULL,
    popu_lsa integer NOT NULL,
    popu_und integer NOT NULL,
    centlib integer NOT NULL,
    branlib integer NOT NULL,
    bkmob integer NOT NULL,
    totstaff numeric(8,2) NOT NULL,
    bkvol integer NOT NULL,
    ebook integer NOT NULL,
    audio_ph integer NOT NULL,
    audio_dl integer NOT NULL,
    video_ph integer NOT NULL,
    video_dl integer NOT NULL,
    ec_lo_ot integer NOT NULL,
    subscrip integer NOT NULL,
    hrs_open integer NOT NULL,
    visits integer NOT NULL,
    reference integer NOT NULL,
    regbor integer NOT NULL,
    totcir integer NOT NULL,
    kidcircl integer NOT NULL,
    totpro integer NOT NULL,
    gpterms integer NOT NULL,
    pitusr integer NOT NULL,
    wifisess integer NOT NULL,
    obereg text NOT NULL,
    statstru text NOT NULL,
    statname text NOT NULL,
    stataddr text NOT NULL,
    longitude numeric(10,7) NOT NULL,
    latitude numeric(10,7) NOT NULL
);

 * sqlite:///dbs/w02/library.db
Done.
Done.
Done.
Done.


[]

**And now**, let's fill the tables with data from our CSV files.

In [8]:
db = "dbs/w02/library.db"
csv = "dbs/w02/pls_fy2016_libraries.csv"
table = "libraries_2016"

sqlite_command = f"""
.mode csv
.import '{csv}' {table}
"""
subprocess.run(["sqlite3", db], input=sqlite_command, text=True)

# 2017

db = "dbs/w02/library.db"
csv = "dbs/w02/pls_fy2017_libraries.csv"
table = "libraries_2017"

sqlite_command = f"""
.mode csv
.import '{csv}' {table}
"""
subprocess.run(["sqlite3", db], input=sqlite_command, text=True)

CompletedProcess(args=['sqlite3', 'dbs/w02/library.db'], returncode=0)

**Let's check** the rows count for each of the tables now.

In [9]:
%%sql
    SELECT 2016 AS 'Year', count(*) AS NumRows FROM libraries_2016
UNION ALL
    SELECT 2017, count(*) AS NumRows FROM libraries_2017
UNION ALL
    SELECT 2018, count(*) AS NumRows FROM libraries_2018;

 * sqlite:///dbs/w02/library.db
Done.


Year,NumRows
2016,9253
2017,9246
2018,9262


## 🧮 1. SQL Aggregation & Grouping (with SQLite Notes)

SQL lets you **summarize** data using **aggregate functions**, and **group** it with GROUP BY. This is especially helpful for statistics, reports, and dashboards.

### 🎯 **Common Aggregate Functions**

| Function    | Description          | Example              |
|-------------|----------------------|----------------------|
| `COUNT()`   | Number of rows       | `COUNT(*)`           |
| `SUM()`     | Total of values      | `SUM(amount)`        |
| `AVG()`     | Mean average         | `AVG(score)`         |
| `MIN()`     | Smallest value       | `MIN(age)`           |
| `MAX()`     | Largest value        | `MAX(salary)`        |

:::{.callout-note}
✅ **SQLite supports all these functions.**
:::

### 🧩 Basic Grouping Example

**Q:** How many students are in each major?

```sql
SELECT major, COUNT(*) AS student_count
FROM students
GROUP BY major;
```

:::{.callout-note}
✅ SQLite allows this even if you’re selecting columns not in the GROUP BY clause or not inside an aggregate function. It will return one arbitrary value from each group for such columns.
:::

**Example (SQLite accepts this):**

```sql
SELECT major, name
FROM students
GROUP BY major;
```

:::{.callout-note}
This works in SQLite — it will return one name for each major, but which name is returned is undefined.
❌ PostgreSQL (and other standards-compliant SQL engines) will **reject** this query with an error unless name is also in the GROUP BY or wrapped in an aggregate like MIN(name).
:::

### 🔍 Filtering Groups with HAVING

Use HAVING to filter after grouping.

**Q:** Show only majors with more than 10 students:

```sql
SELECT major, COUNT(*) AS student_count
FROM students
GROUP BY major
HAVING COUNT(*) > 10;
```

:::{.callout-note}
ℹ️ In SQLite, you can refer to the alias (student_count) in HAVING.
In PostgreSQL, you’d need to repeat the expression: HAVING COUNT(*) > 10.
:::

### 📊 Combining Aggregates

**Q:** Show average GPA per department — only if it’s above 3.5:

```sql
SELECT department, AVG(gpa) AS avg_gpa
FROM students
GROUP BY department
HAVING AVG(gpa) > 3.5;
```

### 🔗 Grouping by Expressions

You can group by computed values like substrings or date parts.

**Q:** Count students by **admission year** (from text-based dates):

```sql
SELECT SUBSTR(admission_date, 1, 4) AS year, COUNT(*) AS num_students
FROM students
GROUP BY year;
```

The SUBSTR() function extracts a **substring** from a given string.

```sql
SUBSTR(string, start, length)
```
* string: the text to extract from
* start: the position to start (1-based index)
* length (optional): how many characters to return

```sql
-- Get the first 4 characters of a date
SELECT SUBSTR('2024-09-15', 1, 4);  -- Returns '2024'

-- Get the first letter of a name
SELECT SUBSTR(name, 1, 1) FROM students;
```

📌 If length is omitted, SQLite returns the rest of the string from start.

:::{.callout-note}
✅ Works well in SQLite (which stores dates as text: YYYY-MM-DD).
❗ In PostgreSQL, use: EXTRACT(YEAR FROM admission_date).
:::

### 🧠 Tips for Grouping Queries

* Use GROUP BY with aggregate functions.
* Use HAVING to filter groups (not rows).
* Use WHERE for filtering before grouping.
* Prefer column names or expressions in GROUP BY, not just positions (e.g., avoid GROUP BY 1).
* Be cautious using non-aggregated, non-grouped columns in SQLite — it’s allowed, but not portable or predictable.

## 1.1 Counting Distinct Values in a Column

We can combine `DISTINCT` with combinations of values like `count()`. This will return a count of distinct values from a column.

In [10]:
%%sql
SELECT count(libname)
FROM libraries_2017;

 * sqlite:///dbs/w02/library.db
Done.


count(libname)
9246


In [12]:
%%sql
SELECT count(DISTINCT libname)
FROM libraries_2017;

 * sqlite:///dbs/w02/library.db
Done.


count(DISTINCT libname)
8456
